In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tqdm
import torch 
import torch.nn.functional as func

import dataset
import models

FAUST = "../../Downloads/Mesh-Datasets/MyFaustDataset"
MODEL_PATH = "../model_data/data.pt"

dataset = dataset.FaustDataset(FAUST)

model = models.ChebClassifier(
    param_conv_layers=[64,64,32,32],
    E_t=dataset.downscaled_edges, 
    D_t=dataset.downscale_matrices,
    num_classes = 10)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
model.load_state_dict(torch.load(MODEL_PATH))


In [ ]:
import plotly
import plotly.io as pio
import plotly.graph_objects as go

def visualize(pos, face, intensity=None):
    """
    visualize input mesh
    """
    
    pos = pos.numpy()
    f = face.numpy()
    
    if intensity is None:
        intensity = np.ones(pos.shape[0])
    
    x,y,z = pos[:,0], pos[:,1], pos[:,2]
    i,j,k = f[0,:], f[1,:], f[2,:]
    mesh = go.Mesh3d(x=x, y=z, z=y,
              color='lightpink',
              intensity=intensity,
              opacity=1,
              i=i, j=j, k=k,
              showscale=True)
    layout = go.Layout(scene=go.layout.Scene(aspectmode="data")) 

    #pio.renderers.default="plotly_mimetype"
    fig = go.Figure(data=[mesh],
                   layout=layout)
    fig.update_layout(
        autosize=True,
        margin=dict(l=20, r=20, t=20, b=20),
        paper_bgcolor="LightSteelBlue")
    fig.show()
    

now that we have defined the visualization procedure, we can show some meshes:

In [ ]:
import mesh.transforms as mo

mesh = dataset[0]
x = mesh.pos.clone()
mo.transform_rotation_(mesh.pos,dims=[1,2,3,4,2])
visualize(mesh)

In [ ]:
from mesh.laplacian import laplacebeltrami_FEM

mesh = dataset[0]
pos = dataset[2].pos
face = dataset[0].face.t()
stiff, area, lump = laplacebeltrami_FEM(pos, face)

f = torch.zeros([pos.shape[0],1])
f[0] = 1
f = lump.view(-1,1)*torch.sparse.mm(stiff, f)
f = f.numpy()

visualize(pos,face.t(), intensity=f)

# Eigenvalues filters

In [ ]:
import numpy as np
import torch 
import torch_sparse as tsparse

import dataset
import mesh.laplacian

from scipy.sparse import linalg as sla
from scipy import linalg as la

ds_i=0
data = dataset.FaustDataset("../../Downloads/Mesh-Datasets/MyFaustDataset")
pos = data[0].pos
for i in range(ds_i): 
    pos = data.downscale_matrices[i].matmul(pos)
faces = data.downscaled_faces[ds_i]


In [ ]:
n=pos.shape[0]

#(si,sv), (ai,av) = mesh.laplacian.LB_v2(pos, faces.t())
#li,lv = tsparse.spspmm(ai, 1/av, si,sv, m=n,k=n,n=n)

pos = pos.to(torch.double)
S,A,_ = mesh.laplacian.laplacebeltrami_FEM(pos, faces.t())
S = S.to_dense()
A = A.to_dense()
L = torch.inverse(A).matmul(S)

#values = lv.detach().cpu().numpy()
#indices = li.detach().cpu().numpy()
#L = scisparse.coo_matrix((values, indices)).tocsc()
#e, phi = sla.eigsh(Ascipy, k=100)

e, phi = la.eigh(a=S,b=A)


phi = torch.tensor(phi[:,:1500])
e = torch.tensor(e[:1500])

import matplotlib.pyplot as plt
plt.plot(range(len(e)), e)
visualize(pos, faces, intensity=phi[:,1])


In [ ]:
f = torch.zeros([pos.shape[0],1],dtype=pos.dtype)
f[10] = 100

x_hat = phi.t().matmul(f)
x_hat = e.view(-1,1)*x_hat
dirchlet1 = phi.matmul(x_hat)
dirchlet2 = L@f
tmp = torch.norm(L@pos, dim=1, p=2)


visualize(pos, faces, tmp)

In [ ]:
pos = pos.to(torch.double)
n = pos.shape[0]
r = torch.normal(0, std=1e-5, size=[n, 3], requires_grad=True, device=pos.device)
optimizer = torch.optim.Adam([r], lr=8e-6, weight_decay=5e-4)

visualize(r.clone().detach(), faces, tmp)

for i in tqdm.tqdm(range(1000)):
    optimizer.zero_grad()
    (si,sv), (ai,av) = mesh.laplacian.LB_v2(r, faces.t())
    _,L1 = tsparse.spspmm(ai, av.pow_(-1), si,sv, n,n,n)
    
    loss = ((L1)**2).sum()
    loss.backward()
    optimizer.step()
visualize(r.clone().detach(), faces, tmp)

